# RBM for Collaborative Filtering

## Activation of visible layer


In [1]:
import tensorflow as tf
import numpy as np

from rbm.util.util import softmax, Σ

tf.set_random_seed(42)
session = tf.InteractiveSession()

### Parameters

In [127]:
movies = 1
ratings = 10
users = 2

visible_size = movies*ratings
hidden_size = 5
visible_shape = [users, visible_size]

In [128]:
users_ratings = np.array((
    (1), # User 1
    (6)  # User 2
))

def assign_rating(value):
    response = [0]*ratings
    response[value] = 1.
    return response

# Hidden and visible values
data = tf.constant(
    np.array(list(map(assign_rating, users_ratings.reshape([-1])))).reshape(visible_shape),
    dtype=tf.float32
).T

#print('User ratings')
#tf.reshape(data.T, [-1, movies, ratings]).eval()

data = data.eval()
data[0][1] = 1
data[0][0] = 1
data[5][1] = 1
data[4][1] = 1
data[8][0] = 1
data[6][0] = 1
data

array([[1., 1.],
       [1., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [1., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.]], dtype=float32)

In [129]:
# Params values
W = 1 * tf.random_normal([hidden_size, visible_size], dtype=tf.float32)
b_v = 0.01 * tf.random_normal([visible_size, 1], dtype=tf.float32)
b_h = 0.01 * tf.random_normal([hidden_size, 1], dtype=tf.float32)

In [133]:
from tensorflow.contrib.distributions import Bernoulli

σ = tf.sigmoid

probabilities = σ(W @ data + b_h)
h = tf.cast(Bernoulli(probs=probabilities).sample(), tf.float32)

print('Example of h sampled: 3x2 because are 3 hidden units and the minibatch size is 2')
h.eval()

Example of h sampled: 3x2 because are 3 hidden units and the minibatch size is 2


array([[1., 1.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [1., 0.]], dtype=float32)

### Probability
https://github.com/thepapercoder/RBM/blob/master/rbm.py#L147-L153

In [6]:
x_sigmoid = h.T @ W + b_v.T

# Remove all the movie have not been rating
# In really, remove all the movie-rating probabilities
mask = tf.sign(data)
x_maked = x_sigmoid * mask.T

# Reshape for apply softmax correctly
x = tf.reshape(x_maked, [users, -1, ratings])

# Calc probability
probabilies = softmax(x)

probabilies = tf.reshape(probabilies, [users, -1])

### Sample

In [7]:
samples = tf.nn.relu(tf.sign(probabilies - tf.random_uniform(tf.shape(probabilies))))

print('The generate samples does not respect the restriction')
tf.reshape(samples, [users, -1, ratings]).eval()

The generate samples does not respect the restriction


array([[[0., 1., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.]],

       [[0., 1., 0.],
        [1., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)

## A simple test with argmax

In [8]:
probabilies_calculated = probabilies.eval()

c = x_sigmoid.eval()

print(tf.reshape(c, [users, -1, ratings]).eval())
tf.argmax(tf.reshape(c, [users, -1, ratings]), axis=2).eval()

[[[ 2.6223373   1.0029061  -0.6231793 ]
  [ 1.2314824   0.87618446  4.7124147 ]
  [ 0.41593808 -0.62558526  0.9319977 ]
  [ 0.27236003 -0.46953848  0.96402305]]

 [[ 2.3110352   0.7221107  -0.702591  ]
  [ 2.8766577   0.34879747  4.1321445 ]
  [ 0.2486911  -0.09267695  0.8595692 ]
  [ 0.7815393  -0.18940184 -0.85840446]]]


array([[0, 2, 2, 2],
       [0, 2, 2, 0]])

## Probability

In [134]:
print('Expectation parameter')
x_sigmoid = h.T @ W + b_v.T
x_sigmoid = x_sigmoid.eval()
print(x_sigmoid)
print()

# Reshape for apply softmax correctly
print('Parameter reshape to apply probability')
total_of_users = -1
x = tf.reshape(x_sigmoid, [total_of_users, movies, ratings])
print(x.eval())
print()

print('Softmax probability')
# Calc probability
probability = softmax(x)
probability.eval()
#probabilies = tf.reshape(probabilies, [users, -1])

Expectation parameter
[[-0.5922705   0.40584487 -0.63715506  0.05796758  0.11270514 -0.6032788
   1.4189985  -0.08548038 -0.41111588  1.7974641 ]
 [ 0.39257938 -1.0894061  -1.3418009   1.355403    0.5096347   0.89495105
  -0.24924284  0.08597186 -0.44713634 -0.5337291 ]]

Parameter reshape to apply probability
[[[-0.5922705   0.40584487 -0.63715506  0.05796758  0.11270514
   -0.6032788   1.4189985  -0.08548038 -0.41111588  1.7974641 ]]

 [[ 0.39257938 -1.0894061  -1.3418009   1.355403    0.5096347
    0.89495105 -0.24924284  0.08597186 -0.44713634 -0.5337291 ]]]

Softmax probability


array([[[0.03242535, 0.08797529, 0.03100214, 0.06212687, 0.06562234,
         0.03207036, 0.24230799, 0.05382461, 0.03886504, 0.35378   ]],

       [[0.11248957, 0.02555608, 0.01985548, 0.29461932, 0.1264587 ,
         0.18590432, 0.05920692, 0.08278551, 0.04857673, 0.04454732]]],
      dtype=float32)

In [136]:
#h = tf.cast(x, tf.float32)
sample = Bernoulli(probs=probability).sample().eval()
sample = tf.cast(sample, tf.float32)

sample.eval()
#Bernoulli(probs=[0., 0.5, 0.]).sample().eval()
#bernoulli_sample(p=probabilities)

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [106]:
shape = [-1, visible_size]
tf.reshape(sample, shape).T.eval()

array([[1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.]], dtype=float32)

## Expectation

In [10]:
x = tf.constant([[
    [.2, .5, .3],
    [.1, .1, .8],
    [.1, .1, .8],
    [.1, .1, .8],
], [
    [.7, .2, .1],
    [.1, .1, .8],
    [.7, .2, .1],
    [.1, .1, .8],
]])

axis = 2
size = 3

ratings_range = tf.range(1, 3+1, dtype=tf.float32)

Σ = tf.reduce_sum(x * ratings_range, axis=axis)
print(Σ.eval())
expectation = tf.cast(tf.round(Σ), tf.int32).eval()
expectation

[[2.1000001 2.7       2.7       2.7      ]
 [1.4000001 2.7       1.4000001 2.7      ]]


array([[2, 3, 3, 3],
       [1, 3, 1, 3]], dtype=int32)

In [11]:
tf.one_hot(expectation-1, depth=3).eval()

array([[[0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]],

       [[1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.]]], dtype=float32)

## Top k

In [12]:
example = probability.eval()
print(example)

values, index = tf.nn.top_k(example, k=2, sorted=False, name=None)
index.eval()

[[[0.12584028 0.7189191  0.15524063]
  [0.07883426 0.725881   0.1952848 ]
  [0.28008157 0.43409774 0.28582066]
  [0.32059222 0.24171223 0.43769553]]

 [[0.07457197 0.01979918 0.9056288 ]
  [0.16013639 0.71088207 0.12898158]
  [0.7697683  0.13352703 0.09670465]
  [0.03156377 0.93171567 0.03672057]]]


array([[[2, 1],
        [2, 1],
        [2, 1],
        [0, 2]],

       [[0, 2],
        [0, 1],
        [1, 0],
        [2, 1]]], dtype=int32)

In [13]:
result = tf.one_hot(index, depth=3).eval()
print(result)

tf.reduce_sum(result, axis=2).eval()

[[[[0. 0. 1.]
   [0. 1. 0.]]

  [[0. 0. 1.]
   [0. 1. 0.]]

  [[0. 0. 1.]
   [0. 1. 0.]]

  [[1. 0. 0.]
   [0. 0. 1.]]]


 [[[1. 0. 0.]
   [0. 0. 1.]]

  [[1. 0. 0.]
   [0. 1. 0.]]

  [[0. 1. 0.]
   [1. 0. 0.]]

  [[0. 0. 1.]
   [0. 1. 0.]]]]


array([[[0., 1., 1.],
        [0., 1., 1.],
        [0., 1., 1.],
        [1., 0., 1.]],

       [[1., 0., 1.],
        [1., 1., 0.],
        [1., 1., 0.],
        [0., 1., 1.]]], dtype=float32)